# Set up your environment

Use the mlgeo enviroment from class

Use `ClusTremor-main/requirements.txt` for additional setup

In [ ]:
import numpy as np
import h5py
import os, re, glob
import math
from scipy import signal
from scipy.signal import butter, lfilter
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from matplotlib.pyplot import savefig
import seaborn as sns

import tensorflow as tf
from tensorflow.keras import regularizers
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D, Conv1D, MaxPooling1D, UpSampling1D, Flatten, Dropout, Reshape 
from tensorflow.keras.layers import Bidirectional, BatchNormalization, ZeroPadding1D, Conv2DTranspose
from tensorflow.keras.layers import Layer, InputSpec
from tensorflow.keras.models import Model

from tensorflow.keras.optimizers import SGD, Adam, schedules

from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, ReduceLROnPlateau, EarlyStopping, CSVLogger
from tensorflow.keras.initializers import VarianceScaling

sns.set_style('darkgrid')
sns.set_palette('muted')

sns.set_style('darkgrid')
sns.set_palette('muted')

# Load data and split train-test

In [ ]:
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset

# Step 1: Load data
print("Beginning to load file...")

# Load the seismic data (adjust file path as needed)
data_file = np.load("Input.npy", mmap_mode='r', allow_pickle=True)

print("File loaded")

# Step 2: Split data into train and test sets
print("Split data into training and testing sets")
train_data, test_data = train_test_split(data_file, test_size=0.2, random_state=46, shuffle=True)

print('Size of data_file:', data_file.shape)
print('Size of train set:', train_data.shape)
print('Size of test set:', test_data.shape)

# Step 3: Convert the data into PyTorch tensors
train_data_tensor = torch.tensor(train_data, dtype=torch.float32)
test_data_tensor = torch.tensor(test_data, dtype=torch.float32)

# If the data has more dimensions, you can reshape or modify it as needed, but
# this will depend on the shape of your `Input.npy` file.

# Example (assuming 4D input with channels, height, and width):
# train_data_tensor = train_data_tensor.view(-1, 1, height, width)
# test_data_tensor = test_data_tensor.view(-1, 1, height, width)

# Step 4: Prepare DataLoader for batching
batch_size = 32

# Create TensorDataset and DataLoader for training and testing sets
train_dataset = TensorDataset(train_data_tensor, train_data_tensor)  # (input, target) both are the same for autoencoder
test_dataset = TensorDataset(test_data_tensor, test_data_tensor)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Data is now ready for training in PyTorch

# Autoencoder architecture construction

Import the dependencies first!
!pip install torch
!pip install torchinfo

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchinfo import summary

# Set the seed for reproducibility
sd = 46
torch.manual_seed(sd)

# Function to calculate "same" padding for strided convolutions
def calculate_same_padding(kernel_size, stride):
    return ((stride - 1) + (kernel_size - 1)) // 2

# Define the autoencoder model
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()

        # Encoder layers with manually calculated "same" padding
        self.conv1 = nn.Conv2d(1, 8, kernel_size=(7, 5), stride=2, padding=(calculate_same_padding(7, 2), calculate_same_padding(5, 2)))
        self.conv2 = nn.Conv2d(8, 16, kernel_size=(5, 3), stride=2, padding=(calculate_same_padding(5, 2), calculate_same_padding(3, 2)))
        self.conv3 = nn.Conv2d(16, 32, kernel_size=(5, 3), stride=2, padding=(calculate_same_padding(5, 2), calculate_same_padding(3, 2)))
        self.conv4 = nn.Conv2d(32, 64, kernel_size=(5, 3), stride=2, padding=(calculate_same_padding(5, 2), calculate_same_padding(3, 2)))

        # Fully connected layers
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(64 * 6 * 8, 24)  # Adjusted based on conv output
        self.fc2 = nn.Linear(24, 3072)

        # Decoder layers
        self.fc3 = nn.Linear(3072, 64 * 6 * 8)  # Adjust based on the shape before flattening
        self.conv_transpose1 = nn.ConvTranspose2d(64, 32, kernel_size=(5, 3), stride=2, padding=(2, 1), output_padding=(1, 1))  # Adjust output_padding
        self.conv_transpose2 = nn.ConvTranspose2d(32, 16, kernel_size=(5, 3), stride=2, padding=(2, 1), output_padding=(1, 1))
        self.conv_transpose3 = nn.ConvTranspose2d(16, 8, kernel_size=(5, 3), stride=2, padding=(2, 1), output_padding=(1, 1))
        self.conv_transpose4 = nn.ConvTranspose2d(8, 1, kernel_size=(7, 5), stride=2, padding=(3, 2), output_padding=(1, 1))  # Adjust padding/output_padding

    def forward(self, x):
        # Encoder
        x = F.elu(self.conv1(x))
        x = F.elu(self.conv2(x))
        x = F.elu(self.conv3(x))
        x = F.elu(self.conv4(x))

        # Save shape for later use in decoder
        shape_before_flattening = x.shape

        # Fully connected layers
        x = self.flatten(x)
        encoded = F.elu(self.fc1(x))
        x = F.elu(self.fc2(encoded))

        # Decoder
        x = F.elu(self.fc3(x))
        x = x.view(-1, 64, 6, 8)  # Reshape to the shape before flattening
        x = F.elu(self.conv_transpose1(x))
        x = F.elu(self.conv_transpose2(x))
        x = F.elu(self.conv_transpose3(x))
        decoded = self.conv_transpose4(x)  # No activation for output layer

        return decoded, encoded


# Create model instance
autoencoder = Autoencoder()

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move the model to the device
autoencoder.to(device)

# Ensure that the input data is on the same device (for example, random input)
dummy_input = torch.randn(1, 1, 96, 128).to(device)  # Simulate a batch of size 1

# Now use torchinfo for the summary
summary(autoencoder, input_size=(1, 1, 96, 128))


# Initial training phase: Pretraining the autoencoder

In [ ]:
import torch
import torch.optim as optim
import torch.nn as nn
from torch.optim.lr_scheduler import ExponentialLR
import pandas as pd
import time

start_time = time.time()

# Define constants
NUM_OF_EPOCHS = 50
initial_lr = 0.001
decay_steps = 1000
decay_rate = 0.5
patience = 30

# Initialize model, optimizer, and scheduler
model = Autoencoder().to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=initial_lr)
scheduler = ExponentialLR(optimizer, gamma=decay_rate**(1 / decay_steps))

# Setup CSV logger
csv_log = {'epoch': [], 'train_loss': [], 'val_loss': [], 'learning_rate': []}

# Early stopping parameters
best_val_loss = float('inf')
early_stop_counter = 0

# Training loop with early stopping and logging
for epoch in range(NUM_OF_EPOCHS):
    # Training phase
    model.train()
    train_loss = 0
    
    for data, _ in train_loader:
        
        data = data.squeeze().unsqueeze(1)  # Remove the extra dimension
        inputs = data.to(device)
        optimizer.zero_grad()
        outputs, _ = model(inputs)
        loss = criterion(outputs, inputs)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    
    # Validation phase
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for data, _ in test_loader:
            
            #data = data.unsqueeze(1)
            data = data.squeeze().unsqueeze(1)  # Ensure correct shape
            inputs = data.to(device)
            outputs, _ = model(inputs)
            loss = criterion(outputs, inputs)
            val_loss += loss.item()

    # Average losses
    train_loss /= len(train_loader)
    val_loss /= len(test_loader)

    # Log values
    current_lr = scheduler.get_last_lr()[0]
    csv_log['epoch'].append(epoch + 1)
    csv_log['train_loss'].append(train_loss)
    csv_log['val_loss'].append(val_loss)
    csv_log['learning_rate'].append(current_lr)

    # Early stopping check
    if val_loss < best_val_loss:
        print(f'Val loss decreased from {best_val_loss:.4f} to {val_loss:.4f}')
        best_val_loss = val_loss
        early_stop_counter = 0
    else:
        early_stop_counter += 1
        if early_stop_counter >= patience:
            print(f"Early stopping at epoch {epoch+1}")
            break

    # Print progress
    print(f"Epoch {epoch+1}/{NUM_OF_EPOCHS}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, LR: {current_lr:.6f}")

    # Step the learning rate scheduler
    scheduler.step()

# Save the CSV log
pd.DataFrame(csv_log).to_csv('pretrain_log.csv', index=False)

end_time = time.time()
pretraining_ac_run_time = end_time - start_time
print(f"Training completed in {pretraining_ac_run_time:.2f} seconds")


# Evaluate autoencoder reconstruction

In [ ]:
import pandas as pd
df = pd.read_csv('pretrain_log.csv')
fig= plt.figure(figsize=(7, 5))
plt.plot(df['epoch'],df['train_loss'], color='b',label='Training Loss', linewidth=3.0)
plt.plot(df['epoch'],df['val_loss'], color='darkorange',label='Validation Loss', linewidth=3.0)

plt.ylabel('Loss', fontsize= 18)
plt.xlabel('Epoch', fontsize= 18)
plt.title('Reconstruction loss of the autoencoder', fontsize= 18)
plt.yticks (fontsize= 18)
plt.xticks (fontsize= 18)

plt.legend(loc= 1, frameon= False, fontsize= 18)
plt.tight_layout()
plt.show ()
fig.savefig ( 'ReconstructionLoss.png', dpi= 100)

In [ ]:
# Filter the DataFrame to start from epoch 3
df_filtered = df[df['epoch'] >= 3]

# Plot the training and validation loss
fig = plt.figure(figsize=(7, 5))
plt.plot(df_filtered['epoch'], df_filtered['train_loss'], color='b', label='Training Loss', linewidth=3.0)
plt.plot(df_filtered['epoch'], df_filtered['val_loss'], color='darkorange', label='Validation Loss', linewidth=3.0)

plt.ylabel('Loss', fontsize=18)
plt.xlabel('Epoch', fontsize=18)
plt.title('Reconstruction loss of the autoencoder (Starting from Epoch 3)', fontsize=18)
plt.yticks(fontsize=18)
plt.xticks(fontsize=18)

plt.legend(loc=1, frameon=False, fontsize=18)
plt.tight_layout()
plt.show()
fig.savefig('ReconstructionLoss_from_epoch_3.png', dpi=100)

In [ ]:
# Save the model's state_dict
torch.save(model.state_dict(), 'autoencoder-model.pth')

# Re-import or re-define the Autoencoder class
autoencoder = Autoencoder()  # Instantiate the model
autoencoder.load_state_dict(torch.load('autoencoder-model.pth'))  # Load the state_dict

# Move to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
autoencoder.to(device)

# Set the model to evaluation mode if you're testing or inferring
autoencoder.eval()


In [ ]:
data_tensor = torch.tensor(data_file)  # Convert to PyTorch tensor
data_tensor = data_tensor.unsqueeze(1)  # Adds a dimension at position 1

data_tensor.shape

### Autoencoder input-output visualization

In [ ]:
decoded_imgs, _ = autoencoder(data_tensor.float().to(device))



import librosa
import librosa.display

s = 300
e= 305
n = e-s

plt.figure(figsize=(20, 4))
for i in range(s,e):
    
    # Display original
    ax = plt.subplot(2, n, i-s + 1)
    librosa.display.specshow(data_file[i,:,:], alpha=None, cmap='hot', antialiased=True)
    plt.colorbar ()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # Display reconstruction
    ax = plt.subplot(2, n, i-s + 1 + n)
    librosa.display.specshow(decoded_imgs.cpu().detach().numpy()[i,0, :,:], alpha=None, cmap='hot', antialiased=True)
    plt.colorbar ()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

## Kmeans clustering based on extracted features from the autoencoder

In [ ]:
decoded_imgs, encoded_imgs = autoencoder(data_tensor.float().to(device))

kmeans = KMeans(n_clusters=1, random_state=46, n_init=20).fit(encoded_imgs.cpu().detach().numpy())
y = kmeans.predict(encoded_imgs.cpu().detach().numpy())

def plotter(S, y):
    '''
    function to visualize the outputs of t-SNE
    '''
    
    lw = 2
    # create a scatter plot.
    f = plt.figure(figsize=(22, 10))
    ax = f.add_subplot(111)
    plt.scatter(S[y == 0, 0], S[y == 0, 1],color='navy', alpha=.5, lw=lw, s=100)
    ax.axis('off')
    ax.axis('tight') 
    plt.show()
    f.savefig ('Tnse-km-n1', dpi= 100,bbox_inches = "tight")

    return f, ax

enc = encoded_imgs.cpu().detach().numpy()

from sklearn.manifold import TSNE
redu = TSNE(random_state=123).fit_transform(enc)
plotter(redu, y) 


TODO: this plot doesn't look like the paper's plot
This will make the following images look different from the paper.

# Determining optimal number of clusters

In [ ]:
from sklearn.metrics import calinski_harabasz_score
cal = []
K = range(2,16)
for k in K:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=20).fit(encoded_imgs.cpu().detach().numpy())
    labelskm = kmeans.predict(encoded_imgs.cpu().detach().numpy())
    cal.append(calinski_harabasz_score(encoded_imgs.cpu().detach().numpy(), labelskm))
fig= plt.figure(figsize=(7, 5))
plt.plot(K, cal, 'bx-')
plt.xlabel('Number of clusters',fontsize= 18)
plt.ylabel('Calinski-Harabasz score',fontsize= 18)
plt.title('Calinski-Harabasz Score Elbow for K-means Clustering',fontsize= 18)
plt.yticks (fontsize= 18)
plt.xticks (fontsize= 18)
plt.axvline(x = 4, color = 'black')
plt.tight_layout()
plt.show()
fig.savefig ( 'Calinski score.png', dpi=100)

# T-sne visualizations of seismic event clusters in feature domain after pretraining

In [ ]:
enc = encoded_imgs.cpu().detach().numpy()
kmeans = KMeans(n_clusters=4, random_state=46, n_init=20).fit(encoded_imgs.cpu().detach().numpy())
y = kmeans.predict(enc)

def plotter(S, y, target_names):
    '''
    function to visualize the outputs of t-SNE
    '''
    # choose a color palette with seaborn.
    colors = [ 'red', 'mediumblue','darkorange','turquoise','lime', 'turquoise', 'darkorange','lawngreen', 'red', 'saddlebrown']
    
    lw = 2
    # create a scatter plot.
    f = plt.figure(figsize=(22, 10))
    ax = f.add_subplot(111)
    for color, i, target_name in zip(colors, [3,0, 1,2], target_names):
        plt.scatter(S[y == i, 0], S[y == i, 1], color=color, alpha=.5, lw=lw, s=100, label=target_name)
    plt.legend(loc='lower left', shadow=False, scatterpoints=1, prop={'size': 26})
    ax.axis('off')
    ax.axis('tight') 
    plt.show()
    f.savefig ('Tnse-km-n4', dpi= 100,bbox_inches = "tight")

    return f, ax

enc = encoded_imgs.cpu().detach().numpy()
from sklearn.manifold import TSNE
redu = TSNE(random_state=123).fit_transform(enc)
target_names = ['cluster 1', 'cluster 2', 'cluster 3', 'cluster 4']
plotter(redu, y, target_names) 


# Loading the pre-trained model

In [ ]:
data_tensor = torch.tensor(data_file)  # Convert to PyTorch tensor
data_tensor = data_tensor.unsqueeze(1).to(device)  # Adds a dimension at position 1


# Assuming autoencoder and data_file are already defined
autoencoder.eval()  # Set to evaluation mode

# Define a dictionary to store layer outputs
layer_outputs = {}

# Function to create hooks for capturing the outputs
def get_layer_output_hook(name):
    def hook(module, input, output):
        layer_outputs[name] = output
    return hook

# Register hooks on each layer you want to capture (starting from layer 1 as in your example)
for idx, layer in enumerate(list(autoencoder.children())[1:], start=1):
    layer.register_forward_hook(get_layer_output_hook(f'layer_{idx}'))

# Run a forward pass with `data_file` to capture the outputs
with torch.no_grad():  # Disable gradient computation
    data = torch.tensor(data_tensor, dtype=torch.float32)  # Convert data_file to tensor
    autoencoder(data)  # Run data through the model to capture intermediate outputs

# Display the outputs
for name, output in layer_outputs.items():
    print(f"{name} output shape: {output.shape}")


# Integrating clustering layer into autoencoder bottleneck

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

# Set seed for reproducibility
seed = 46
torch.manual_seed(seed)
np.random.seed(seed)

# Initialize Xavier initializer (same as GlorotUniform in TensorFlow)
initializer = nn.init.xavier_uniform_
n_clusters = 4 # Set desired number of clusters


# Define the clustering layer in PyTorch
class ClusteringLayer(nn.Module):
    def __init__(self, n_clusters, input_dim, alpha=1.0):
        super(ClusteringLayer, self).__init__()
        self.n_clusters = n_clusters
        self.alpha = alpha
        
        # Initialize the cluster centers
        self.clusters = nn.Parameter(torch.Tensor(n_clusters, input_dim))
        # Define cluster centers as a trainable parameter
        
        initializer(self.clusters)  # Xavier initialization

    def forward(self, inputs):
        
        print(inputs.shape)
        self.clusters = self.clusters.to(inputs.device)
        # Compute the soft assignment q
        q = 1.0 / (1.0 + (torch.sum((inputs.unsqueeze(1) - self.clusters) ** 2, dim=2) / self.alpha))
        q = q ** ((self.alpha + 1.0) / 2.0)
        q = q / torch.sum(q, dim=1, keepdim=True)  # Normalize to get probabilities per cluster
        return q

# Define the full model with the clustering layer on top of the autoencoder
class ClusteredAutoencoder(nn.Module):
    def __init__(self, autoencoder, n_clusters):
        super(ClusteredAutoencoder, self).__init__()
        self.autoencoder = autoencoder
        # Clustering layer connected to the output of the encoder
        self.clustering_layer = ClusteringLayer(n_clusters=n_clusters, input_dim=24)  # Match to `encoded` dimension

    def forward(self, x):
        decoded, encoded = self.autoencoder(x)
        q = self.clustering_layer(encoded)  # Cluster assignments
        return decoded, q  # Return both decoded output and cluster assignments

# Instantiate the models
n_clusters = 4  # Set desired number of clusters
model = ClusteredAutoencoder(autoencoder, n_clusters=n_clusters)

# Define the KL-divergence loss function
def kld_loss(q, p):
    return F.kl_div(q.log(), p, reduction='batchmean')

# Example optimizer setup
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

# Training loop placeholder
def train(model, data_loader, num_epochs=10):
    model.train()
    for epoch in range(num_epochs):
        for inputs, _ in data_loader:
            decoded, q = model(inputs)
            # Target distribution p, here using a sample softmax normalization of q as a placeholder
            p = (q ** 2) / torch.sum(q, dim=0)
            p = p / torch.sum(p, dim=1, keepdim=True)
            
            # Calculate the losses
            reconstruction_loss = F.mse_loss(decoded, inputs)
            clustering_loss = kld_loss(q, p)
            total_loss = 0.1 * clustering_loss + reconstruction_loss
            
            # Backward and optimize
            optimizer.zero_grad()
            total_loss.backward()
            optimizer.step()

            print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss.item():.4f}')

# Visualize the model (can be printed or logged)

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move the model to the device
model.to(device)

# Ensure that the input data is on the same device (for example, random input)
dummy_input = torch.randn(1, 1, 96, 128).to(device)  # Simulate a batch of size 1

# Now use torchinfo for the summary
summary(model, input_size=(1, 1, 96, 128))

In [ ]:
from sklearn.cluster import KMeans
import torch


# Assuming autoencoder and data_file are already defined
data_tensor = torch.tensor(data_file)  # Convert to PyTorch tensor
data_tensor = data_tensor.unsqueeze(1).to(device)  # Adds a dimension at position 1

autoencoder.eval()  # Set to evaluation mode

# Define a dictionary to store layer outputs
layer_outputs = {}

# Function to create hooks for capturing the outputs
def get_layer_output_hook(name):
    def hook(module, input, output):
        layer_outputs[name] = output
    return hook

# Register hooks on each layer you want to capture (starting from layer 1 as in your example)
for idx, layer in enumerate(list(autoencoder.children())[1:], start=1):
    layer.register_forward_hook(get_layer_output_hook(f'layer_{idx}'))

# Run a forward pass with `data_file` to capture the outputs
with torch.no_grad():  # Disable gradient computation
    data = data_tensor.float()  # Ensure data is in float32 format
    autoencoder(data)  # Run data through the model to capture intermediate outputs

# Display the outputs
for name, output in layer_outputs.items():
    print(f"{name} output shape: {output.shape}")

n_clusters = 4  # Define the number of clusters

# Step 1: Perform KMeans on the encoded output to initialize cluster centers
kmeans = KMeans(n_clusters=n_clusters, random_state=46, n_init=20)
encoded_output = layer_outputs['layer_5'].cpu().detach().numpy()  # Assuming this is the encoded output
y_pred = kmeans.fit_predict(encoded_output)
y_pred_last = np.copy(y_pred)

# Step 2: Set the KMeans cluster centers as initial weights in the clustering layer
cluster_centers = torch.tensor(kmeans.cluster_centers_, dtype=torch.float32, device = device)

# Assuming model.clustering_layer is the clustering layer in the PyTorch model
model.clustering_layer.clusters.data = cluster_centers  # Directly assign cluster centers

# Verify the initialization
print("Number of clusters in KMeans model:", kmeans.n_clusters)
print("Number of clusters in clustering layer:", model.clustering_layer.n_clusters)
print("Assigned KMeans cluster centers as initial weights in the clustering layer.")


In [ ]:
decoded_imgs  = model(data_tensor.float().to(device))[0]

import librosa
import librosa.display

s = 300
e= 305
n = e-s

plt.figure(figsize=(20, 4))
for i in range(s,e):
    
    # Display original
    ax = plt.subplot(2, n, i-s + 1)
    librosa.display.specshow(data_file[i,:,:], alpha=None, cmap='hot', antialiased=True)
    plt.colorbar ()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # Display reconstruction
    ax = plt.subplot(2, n, i-s + 1 + n)
    librosa.display.specshow(decoded_imgs.cpu().detach().numpy()[i,0, :,:], alpha=None, cmap='hot', antialiased=True)
    plt.colorbar ()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

# Finetuning pre-trained model parameters

In [ ]:
import torch
import numpy as np

model.eval()

# Parameters for fine-tuning
batch_size = 32
tol = 0.0001  # tolerance threshold for early stopping
loss_value = 0
index = 0
maxiter = 300
update_interval = 200
index_array = np.arange(data_file.shape[0])
y_pred_last = np.zeros(data_file.shape[0])

data_tensor = torch.tensor(data_file, dtype=torch.float32)
data_tensor = data_tensor.unsqueeze(1).to(device)  # Adds a dimension at position 1

# Define the target distribution function
def target_distribution(q):
    weight = q ** 2 / np.sum(q, axis=0)
    return (weight.T / np.sum(weight, axis=1)).T

# Fine-tuning loop
for ite in range(maxiter):
    print(f"Iteration: {ite}")
    
    # Update clustering target distribution periodically
    if ite % update_interval == 0:
        print("Updating target distribution")
        
        # Forward pass to get cluster assignments
        with torch.no_grad():
            
            
            _, q = model(data_tensor)
       
            q = q.cpu().numpy()  # Convert to numpy for further processing
        
        # Update the target distribution `p`
        p = target_distribution(q)
        y_pred = q.argmax(1)  # Get cluster assignments
        
        # Print loss and clustering information
        print(f"Iter {ite}: ; loss={round(loss_value, 5)}")

        # Check convergence criterion
        delta_label = np.sum(y_pred != y_pred_last).astype(np.float32) / y_pred.shape[0]
        y_pred_last = np.copy(y_pred)
        
        if ite > 0 and delta_label < tol:
            print(f"Convergence achieved: delta_label {delta_label} < tol {tol}")
            break
    
    # Mini-batch training
    start_idx = index * batch_size
    end_idx = min((index + 1) * batch_size, data_file.shape[0])
    idx = index_array[start_idx:end_idx]
    
    batch_data = data_tensor[idx]
    batch_p = torch.tensor(p[idx], dtype=torch.float32, device = device)

    # Zero gradients, forward pass, compute loss, backward pass, and update
    optimizer.zero_grad()
    decoded, q_batch = model(batch_data)
    clustering_loss = F.kl_div(q_batch.log(), batch_p, reduction="batchmean")
    reconstruction_loss = F.mse_loss(decoded, batch_data)
    total_loss = clustering_loss * 0.1 + reconstruction_loss
    total_loss.backward()
    optimizer.step()

    # Update loss and index for next iteration
    loss_value = total_loss.item()
    index = (index + 1) % (data_file.shape[0] // batch_size)


# T-sne visualizations of seismic event clusters in feature domain after finetuning

In [ ]:
autoencoder.eval()  # Set to evaluation mode

# Define a dictionary to store layer outputs
layer_outputs = {}

# Function to create hooks for capturing the outputs
def get_layer_output_hook(name):
    def hook(module, input, output):
        layer_outputs[name] = output
    return hook

# Register hooks on each layer you want to capture (starting from layer 1 as in your example)
for idx, layer in enumerate(list(autoencoder.children())[1:], start=1):
    layer.register_forward_hook(get_layer_output_hook(f'layer_{idx}'))

# Run a forward pass with `data_file` to capture the outputs
with torch.no_grad():  # Disable gradient computation
    data = data_tensor.float()  # Ensure data is in float32 format
    autoencoder(data)  # Run data through the model to capture intermediate outputs

y=y_pred

def plotter(S, y, target_names):
    '''
    function to visualize the outputs of t-SNE
    '''
    # choose a color palette with seaborn.
    colors = ['red', 'mediumblue','darkorange','turquoise', 'lawngreen', 'red', 'saddlebrown']
    
    lw = 2
    # create a scatter plot.
    f = plt.figure(figsize=(22, 10))
    ax = f.add_subplot(111)
    for color, i, target_name in zip(colors, [3,0, 1,2], target_names):
        plt.scatter(S[y == i, 0], S[y == i, 1], color=color, alpha=0.5, lw=lw, s=100, label=target_name)
    plt.legend(loc='lower left', shadow=False, scatterpoints=1, prop={'size': 26})
    ax.axis('off')
    ax.axis('tight') 
    plt.show()
    f.savefig ('Tsne-km-n4-ft.png', dpi= 100,bbox_inches = "tight")
    
    return f, ax

enc = layer_outputs['layer_5'].cpu().detach().numpy()
from sklearn.manifold import TSNE
redu = TSNE(random_state=123).fit_transform(enc)
target_names = [ 'Earthquakes (EQ)','Continuous tremors 1 (CT1)', 'Episodic tremors (ET)', 'Continuous tremors 2 (CT2)' ]
plotter(redu, y, target_names) 


In [ ]:
### Save the labels
np.savetxt('km-n4-ft.txt', y, fmt='%i', delimiter=',')

# Change the order of the cluster numbers (just for a nice representation)

with open('km-n4-ft.txt', 'r') as file :
  filedata = file.read()

# Replace the target string
filedata = filedata.replace('3', 'data_file')
filedata = filedata.replace('2', '3')
filedata = filedata.replace('1', '2')
filedata = filedata.replace('0', '1')
filedata = filedata.replace('data_file', '0')

# Re-write the output
with open('km-n4-ft.txt', 'w') as file:
  file.write(filedata)

### Visualizing cluster changes across time

In [ ]:
from matplotlib.colors import LinearSegmentedColormap

y= np.loadtxt ('km-n4-ft.txt')
fig= plt.figure(figsize=(18, 4))
ax = fig.add_subplot()
x1=list(range(0,2390))
colors = ['red', 'mediumblue','darkorange','turquoise']
cmap_name = 'my_list'
cmap = LinearSegmentedColormap.from_list(cmap_name, colors)
ax.scatter (x1, y, c=y, cmap=cmap, s= 30, alpha=0.2)
ax.set_yticks ([0, 1, 2,3])
ax.set_yticklabels (['EQ','CT1', 'ET','CT2'], fontsize=18)
plt.ylabel('Clusters', fontsize= 18)

ax.set_xticks ([0,175,417,708,996,1117,1425,1782,2142,2390])
ax.set_xticklabels (['12 March','19 March','30 March','15 April','27 April','2 May','15 May','30 May','14 June','24 June'], fontsize=16)
ax.axvline (x=174, linewidth=4, color='black')
ax.axvline (x=996, linewidth=4, color='black')
ax.axvline (x=2127, linewidth=4, color='black')

plt.xlim (0,2390)
#plt.xlim (980,1020)
plt.tight_layout()
fig.savefig ( 'Temporal Cluster Changes.png', dpi= 100)

# Cluster-wise autoencoder input-output visualization

In [ ]:
decoded_imgs[43,:,:,:].shape

In [ ]:
import librosa
import librosa.display
import matplotlib.pyplot as plt
from matplotlib.pyplot import savefig
import matplotlib.gridspec as gridspec


decoded_imgs, _ = autoencoder(data_tensor.float().to(device))
decoded_imgs = decoded_imgs.cpu().detach().numpy()

fig= plt.figure(figsize=(15, 5))
spec = gridspec.GridSpec(2, 3)

ax1 = fig.add_subplot(spec[0, 0])

librosa.display.specshow(data_file[43,:,:], alpha=None, cmap='hot', antialiased=True,y_axis='linear', sr= 8, vmin=-1 , vmax=10)
cbar= plt.colorbar(pad= 0.03)
cbar.ax.tick_params(labelsize=18, rotation=0)
freq =[0,1.33, 2.66, 4]
labelsy = [1,2,3, 4]
plt.yticks (freq, labelsy, fontsize= 18)
ax1.set(ylabel=None)
plt.ylabel('Frequency (Hz)', fontsize= 18)

ax2 = fig.add_subplot(spec[0, 1])

librosa.display.specshow(data_file[44,:,:], alpha=None, cmap='hot', antialiased=True,y_axis='linear', sr= 8, vmin=-1 , vmax=10)
cbar= plt.colorbar(pad= 0.03)
cbar.ax.tick_params(labelsize=18, rotation=0)
freq =[0,1.33, 2.66, 4]
labelsy = [1,2,3, 4]
plt.yticks ([])
ax2.set(ylabel=None)

fig.suptitle ('Cluster EQ', fontsize= 22)

ax3 = fig.add_subplot(spec[0, 2])

librosa.display.specshow(data_file[45,:,:], alpha=None, cmap='hot', antialiased=True,y_axis='linear', sr= 8, vmin=-1 , vmax=10)
cbar= plt.colorbar(pad= 0.03)
cbar.ax.tick_params(labelsize=18, rotation=0)
freq =[0,1.33, 2.66, 4]
labelsy = [1,2,3, 4]
plt.yticks ([])
ax3.set(ylabel=None)

ax4 = fig.add_subplot(spec[1, 0])

librosa.display.specshow(decoded_imgs[43,0,:,:], alpha=None, cmap='hot', antialiased=True,y_axis='linear', sr= 8, vmin=-1 , vmax=10)
cbar= plt.colorbar(pad= 0.03)
cbar.ax.tick_params(labelsize=18, rotation=0)
freq =[0,1.33, 2.66, 4]
labelsy = [1,2,3, 4]
plt.yticks (freq, labelsy, fontsize= 18)
ax4.set(ylabel=None)
labelsx = [0,30,60]
plt.xticks(np.arange(0,128, 63.999), labelsx, fontsize= 18)
plt.ylabel('Frequency (Hz)', fontsize= 18)

ax5 = fig.add_subplot(spec[1, 1])

librosa.display.specshow(decoded_imgs[44,0,:,:], alpha=None, cmap='hot', antialiased=True,y_axis='linear', sr= 8, vmin=-1 , vmax=10)
cbar= plt.colorbar(pad= 0.03)
cbar.ax.tick_params(labelsize=18, rotation=0)
freq =[0,1.33, 2.66, 4]
labelsy = [1,2,3, 4]
plt.yticks ([])
ax5.set(ylabel=None)
ax5.set_xlabel("Time (min)", fontsize= 18)
labelsx = [0,30,60]
plt.xticks(np.arange(0,128, 63.999), labelsx, fontsize=18)

ax6 = fig.add_subplot(spec[1, 2])

librosa.display.specshow(decoded_imgs[45,0,:,:], alpha=None, cmap='hot', antialiased=True,y_axis='linear', sr= 8, vmin=-1 , vmax=10)
cbar= plt.colorbar(pad= 0.03)
cbar.ax.tick_params(labelsize=18, rotation=0)
freq =[0,1.33, 2.66, 4]
labelsy = [1,2,3, 4]
plt.yticks ([])
ax6.set(ylabel=None)
labelsx = [0,30,60]
plt.xticks(np.arange(0,128, 63.999), labelsx, fontsize= 18)

plt.tight_layout()    
plt.savefig ('In&out-EQ.png', dpi=100, bbox_inches = "tight")
plt.show()

In [ ]:
fig= plt.figure(figsize=(15, 5))
spec = gridspec.GridSpec(2, 3)

ax1 = fig.add_subplot(spec[0, 0])

librosa.display.specshow(data_file[353,:,:], alpha=None, cmap='hot', antialiased=True,y_axis='linear', sr= 8, vmin=-1 , vmax=10)
cbar= plt.colorbar(pad= 0.03)
cbar.ax.tick_params(labelsize=18, rotation=0)
freq =[0,1.33, 2.66, 4]
labelsy = [1,2,3, 4]
plt.yticks (freq, labelsy, fontsize= 18)
ax1.set(ylabel=None)
plt.ylabel('Frequency (Hz)', fontsize= 18)

ax2 = fig.add_subplot(spec[0, 1])

librosa.display.specshow(data_file[354,:,:], alpha=None, cmap='hot', antialiased=True,y_axis='linear', sr= 8, vmin=-1 , vmax=10)
cbar= plt.colorbar(pad= 0.03)
cbar.ax.tick_params(labelsize=18, rotation=0)
freq =[0,1.33, 2.66, 4]
labelsy = [1,2,3, 4]
plt.yticks (freq, labelsy, fontsize= 18)
ax2.set(ylabel=None)

fig.suptitle ('Cluster CT1', fontsize= 22)

ax3 = fig.add_subplot(spec[0, 2])

librosa.display.specshow(data_file[355,:,:], alpha=None, cmap='hot', antialiased=True,y_axis='linear', sr= 8, vmin=-1 , vmax=10)
cbar= plt.colorbar(pad= 0.03)
cbar.ax.tick_params(labelsize=18, rotation=0)
freq =[0,1.33, 2.66, 4]
labelsy = [1,2,3, 4]
plt.yticks (freq, labelsy, fontsize= 18)
ax3.set(ylabel=None)

ax4 = fig.add_subplot(spec[1, 0])

librosa.display.specshow(decoded_imgs[353,0,:,:], alpha=None, cmap='hot', antialiased=True,y_axis='linear', sr= 8, vmin=-1 , vmax=10)
cbar= plt.colorbar(pad= 0.03)
cbar.ax.tick_params(labelsize=18, rotation=0)
freq =[0,1.33, 2.66, 4]
labelsy = [1,2,3, 4]
plt.yticks (freq, labelsy, fontsize= 18)
ax4.set(ylabel=None)
labelsx = [0,30,60]
plt.xticks(np.arange(0,128, 63.999), labelsx, fontsize= 18)
plt.ylabel('Frequency (Hz)', fontsize= 18)

ax5 = fig.add_subplot(spec[1, 1])

librosa.display.specshow(decoded_imgs[354,0,:,:], alpha=None, cmap='hot', antialiased=True,y_axis='linear', sr= 8, vmin=-1 , vmax=10)
cbar= plt.colorbar(pad= 0.03)
cbar.ax.tick_params(labelsize=18, rotation=0)
freq =[0,1.33, 2.66, 4]
labelsy = [1,2,3, 4]
plt.yticks (freq, labelsy, fontsize= 18)
ax5.set(ylabel=None)
ax5.set_xlabel("Time (min)", fontsize= 18)
labelsx = [0,30,60]
plt.xticks(np.arange(0,128, 63.999), labelsx, fontsize= 18)

ax6 = fig.add_subplot(spec[1, 2])

librosa.display.specshow(decoded_imgs[355,0,:,:], alpha=None, cmap='hot', antialiased=True,y_axis='linear', sr= 8, vmin=-1 , vmax=10)
cbar= plt.colorbar(pad= 0.03)
cbar.ax.tick_params(labelsize=18, rotation=0)
freq =[0,1.33, 2.66, 4]
labelsy = [1,2,3, 4]
plt.yticks (freq, labelsy, fontsize= 18)
ax6.set(ylabel=None)
labelsx = [0,30,60]
plt.xticks(np.arange(0,128, 63.999), labelsx, fontsize= 18)

plt.tight_layout()    
plt.savefig ('In&out-CT1.png', dpi=100)
plt.show()

In [ ]:
import librosa
import librosa.display
import matplotlib.pyplot as plt
from matplotlib.pyplot import savefig
import matplotlib.gridspec as gridspec

fig= plt.figure(figsize=(15, 5))
spec = gridspec.GridSpec(2, 3)

ax1 = fig.add_subplot(spec[0, 0])

librosa.display.specshow(data_file[1169,:,:], alpha=None, cmap='hot', antialiased=True,y_axis='linear', sr= 8, vmin=-1 , vmax=10)
cbar= plt.colorbar(pad= 0.03)
cbar.ax.tick_params(labelsize=18, rotation=0)
freq =[0,1.33, 2.66, 4]
labelsy = [1,2,3, 4]
plt.yticks (freq, labelsy, fontsize= 18)
ax1.set(ylabel=None)
plt.ylabel('Frequency (Hz)', fontsize= 18)

ax2 = fig.add_subplot(spec[0, 1])

librosa.display.specshow(data_file[1170,:,:], alpha=None, cmap='hot', antialiased=True,y_axis='linear', sr= 8, vmin=-1 , vmax=10)
cbar= plt.colorbar(pad= 0.03)
cbar.ax.tick_params(labelsize=18, rotation=0)
freq =[0,1.33, 2.66, 4]
labelsy = [1,2,3, 4]
plt.yticks (freq, labelsy, fontsize= 18)
ax2.set(ylabel=None)

fig.suptitle ('Cluster ET', fontsize= 22)

ax3 = fig.add_subplot(spec[0, 2])

librosa.display.specshow(data_file[1171,:,:], alpha=None, cmap='hot', antialiased=True,y_axis='linear', sr= 8, vmin=-1 , vmax=10)
cbar= plt.colorbar(pad= 0.03)
cbar.ax.tick_params(labelsize=18, rotation=0)
freq =[0,1.33, 2.66, 4]
labelsy = [1,2,3, 4]
plt.yticks (freq, labelsy, fontsize= 18)
ax3.set(ylabel=None)

ax4 = fig.add_subplot(spec[1, 0])

librosa.display.specshow(decoded_imgs[1169,0,:,:], alpha=None, cmap='hot', antialiased=True,y_axis='linear', sr= 8, vmin=-1 , vmax=10)
cbar= plt.colorbar(pad= 0.03)
cbar.ax.tick_params(labelsize=18, rotation=0)
freq =[0,1.33, 2.66, 4]
labelsy = [1,2,3, 4]
plt.yticks (freq, labelsy, fontsize= 18)
ax4.set(ylabel=None)
labelsx = [0,30,60]
plt.xticks(np.arange(0,128, 63.999), labelsx, fontsize= 18)
plt.ylabel('Frequency (Hz)', fontsize= 18)

ax5 = fig.add_subplot(spec[1, 1])

librosa.display.specshow(decoded_imgs[1170,0,:,:], alpha=None, cmap='hot', antialiased=True,y_axis='linear', sr= 8, vmin=-1 , vmax=10)
cbar= plt.colorbar(pad= 0.03)
cbar.ax.tick_params(labelsize=18, rotation=0)
freq =[0,1.33, 2.66, 4]
labelsy = [1,2,3, 4]
plt.yticks (freq, labelsy, fontsize= 18)
ax5.set(ylabel=None)
ax5.set_xlabel("Time (min)", fontsize= 20)
labelsx = [0,30,60]
plt.xticks(np.arange(0,128, 63.999), labelsx, fontsize= 18)

ax6 = fig.add_subplot(spec[1, 2])

librosa.display.specshow(decoded_imgs[1171,0,:,:], alpha=None, cmap='hot', antialiased=True,y_axis='linear', sr= 8, vmin=-1 , vmax=10)
cbar= plt.colorbar(pad= 0.03)
cbar.ax.tick_params(labelsize=18, rotation=0)
freq =[0,1.33, 2.66, 4]
labelsy = [1,2,3, 4]
plt.yticks (freq, labelsy, fontsize= 18)
ax6.set(ylabel=None)
labelsx = [0,30,60]
plt.xticks(np.arange(0,128, 63.999), labelsx, fontsize= 18)

plt.tight_layout()    
plt.savefig ('In&out-ET.png', dpi=100)
plt.show()